# import

In [2]:
import copy
import csv
import gc
import math
import os
import random as rn
import re
from collections import deque

import numpy as np
import pandas as pd
import tensorflow as tf
from gensim.models import KeyedVectors, word2vec
from IPython.display import HTML, SVG
from keras import backend as K
from keras import (constraints, initializers, layers, models, optimizers,
                   regularizers)
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.engine.topology import Layer
from keras.layers import (LSTM, Activation, Average, Bidirectional, Dense,
                          Dropout, Embedding, Flatten, Input, Lambda, Masking,
                          Permute, Reshape, merge, multiply)
from keras.models import Model, Sequential, load_model, model_from_json
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import model_to_dot, plot_model
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split

pd.set_option('display.max_columns', 100)

# ハイパーパラメータ

In [2]:
LEARNING_RATE = 0.01
LSTM_UNITS = 64
BATCH_SIZE = 16
DROPOUT = 0.1
#NUM_EPOCHS = 10
EPOCHS = 200
ITERATION = 200
#DATA_SIZE = 500000

In [ ]:
word2vec_path = '/hoge/hoge.model'
word2vec_matrix = word2vec.Word2Vec.load(word2vec_path)

In [ ]:
import os
import numpy as np
import tensorflow.python.keras.backend as K
import tensorflow as tf
import random as rn
import json

from sklearn.metrics import roc_curve, auc
from keras.preprocessing import sequence
from keras.backend.tensorflow_backend import set_session

#シードの固定
def seed(seed):
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    tf.compat.v1.keras.backend.set_session(sess)
    os.environ['PYTHONHASHSEED'] = '0'
    np.random.seed(seed)
    rn.seed(seed)
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads = 1, inter_op_parallelism_threads = 1)
    tf.compat.v1.set_random_seed(seed)
    session = tf.compat.v1.Session(graph = tf.compat.v1.get_default_graph(), config = session_conf)
    tf.compat.v1.keras.backend.set_session(session)

# 時間呼び出し

In [ ]:
train_path = '/hoge/hoge.csv'
dev_path = '/hoge/hoge.csv'
test_path = '/hoge/hoge.csv'
df_id_process_time_turn_divide_time_train = pd.read_csv(train_path)
df_id_process_time_turn_divide_time_dev = pd.read_csv(dev_path)
df_id_process_time_turn_divide_time_test = pd.read_csv(test_path)

## 時間をlogに変換

In [6]:
def realtime_to_logtime(x):
    if x == 0:
        return 0
    return math.log(x)

In [7]:
#numpy.ndarrayに変換
log_time_series_train = df_id_process_time_turn_divide_time_train['time'].apply(realtime_to_logtime)

log_time_series_dev = df_id_process_time_turn_divide_time_dev['time'].apply(realtime_to_logtime)

log_time_series_test = df_id_process_time_turn_divide_time_test['time'].apply(realtime_to_logtime)

## レシピの最大手順をリストにする

In [ ]:
train_max_turn_path = '/hoge/hoge.csv'
dev_max_turn_path = '/hoge/hoge.csv'
test_max_turn_path = '/hoge/hoge.csv'

In [ ]:
df_max_turn_train= pd.read_csv(train_max_turn_path, sep=",")

max_turn_train_list = df_max_turn_train.values.tolist()

In [ ]:
df_max_turn_dev= pd.read_csv(dev_max_turn_path, sep=",")

max_turn_dev_list = df_max_turn_dev.values.tolist()

In [ ]:
df_max_turn_test= pd.read_csv(test_max_turn_path, sep=",")

max_turn_test_list = df_max_turn_test.values.tolist()

In [11]:
del df_max_turn_train
del df_max_turn_dev
del df_max_turn_test
gc.collect()

0

## log時間をレシピごとにまとめる

In [12]:
def time_procedure_to_process(log_time_series,max_turn_list):
    process_log_time_list = []
    
    count = 0
    d = deque(max_turn_list)
    #最後のpopが処理されるためにappend
    d.append(0)
    #divideは各手順の最大手順数
    max_turn = d.popleft()
    
    for log_time in log_time_series:
        count += 1
        if max_turn[0] == count:
            process_log_time_list.append(log_time)
            count = 0
            max_turn = d.popleft()
            
    return process_log_time_list

In [13]:
log_time_array_train = np.array(time_procedure_to_process(log_time_series_train,max_turn_train_list))

In [14]:
log_time_array_dev = np.array(time_procedure_to_process(log_time_series_dev,max_turn_dev_list))

In [15]:
log_time_array_test = np.array(time_procedure_to_process(log_time_series_test,max_turn_test_list))

In [16]:
del log_time_series_train
del log_time_series_dev
del log_time_series_test
gc.collect()

20

# 単語にインデックスを割り当てる

In [17]:
word_list = word2vec_matrix.wv.index2word

In [18]:
word_index_dic = {}
index = 1
#unkのindexは０とする
word_index_dic['<unk>'] = 0
for word in word_list:
    if word not in word_index_dic:
        word_index_dic[word] = index
        index += 1

## mecabしたレシピを対応づける

In [ ]:
train_mecab_path = '/hoge/hoge.csv'
dev_mecab_path = '/hoge/hoge.csv'
test_mecab_path = '/hoge/hoge.csv'

In [ ]:
with open(train_mecab_path) as f:
    reader = csv.reader(f)
    mecab_procedure_words_train_list = [row for row in reader]

In [ ]:
with open(dev_mecab_path) as f:
    reader = csv.reader(f)
    mecab_procedure_words_dev_list = [row for row in reader]

In [ ]:
with open(test_mecab_path) as f:
    reader = csv.reader(f)
    mecab_procedure_words_test_list = [row for row in reader]

## 手順をまとめる

In [22]:
def procedure_to_process(mecab_procedure_words_list,max_turn_list):
        process_words_list = []
        process_word_list = []
        count = 0
        d = deque(max_turn_list)
        d.append(0)
        
        max_turn = d.popleft()
        
        
        for procedure_word_list in mecab_procedure_words_list:
            count += 1
            procedure_word_list = deque(procedure_word_list)
            procedure_word_list.popleft()
            procedure_word_list.pop()
            
            if count == max_turn[0]:
                process_word_list.extend(procedure_word_list)
                
                process_word_list.insert(0,'<bop>')
                process_word_list.append('<eop>')
                
                process_words_list.append(process_word_list)
                process_word_list = []
                count = 0
                max_turn = d.popleft()
            else:
                process_word_list.extend(procedure_word_list)
            
        return process_words_list            

In [23]:
mecab_process_words_train_list = procedure_to_process(mecab_procedure_words_train_list,max_turn_train_list)

In [24]:
mecab_process_words_dev_list = procedure_to_process(mecab_procedure_words_dev_list,max_turn_dev_list)

In [25]:
mecab_process_words_test_list = procedure_to_process(mecab_procedure_words_test_list,max_turn_test_list)

In [26]:
del mecab_procedure_words_train_list
del mecab_procedure_words_dev_list
# del mecab_procedure_words_test_list
gc.collect()

20

In [27]:
def get_words_index_list(mecab_procedure_words_list,max_turn_list):
    process_index_list = []
    d = deque(max_turn_list)
    #最後のpopが処理されるためにappend
    d.append(0)
    #divideは各手順の最大手順数
    max_turn = d.popleft()

    for procedure_words_list in mecab_procedure_words_list:
        procedure_index_list = []
  
        for word in procedure_words_list:
            procedure_index_list.append(word_index_dic[word])

        process_index_list.append(procedure_index_list)
    return process_index_list            

In [28]:
word_index_train_list = get_words_index_list(mecab_process_words_train_list,max_turn_train_list)

In [29]:
word_index_dev_list = get_words_index_list(mecab_process_words_dev_list,max_turn_dev_list)

In [30]:
#testデータが元のままでunkをunkとしていなかった
processes_test_list = []
for process in mecab_process_words_test_list:
    process_test_list = []
    for word in process:
        if word in word_list:
            process_test_list.append(word)
        else:
            process_test_list.append('<unk>')
    processes_test_list.append(process_test_list)

In [31]:
word_index_test_list = get_words_index_list(processes_test_list,max_turn_test_list)

# trainだけベクトルを持つembedding_matrixの作成

In [32]:
#valueからkeyを抽出
def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

In [ ]:
size = len(word2vec_matrix[word_list[0]].tolist())

In [34]:
unk_list = np.zeros(size).tolist()

In [ ]:
#train,dev,test全ての単語を使用
#trainに対応しないdev,testの単語は全て０ベクトルとなる
#<unk>も０ベクトルとする
size = len(word2vec_matrix[word_list[0]].tolist())
embedding_matrix = np.zeros((len(word_index_dic)+1, size))
used_index = []
for procedure_index in word_index_train_list:
    for index in procedure_index:
        if index not in used_index:
            word = get_keys_from_value(word_index_dic,index)[0]
            if word == '<unk>':
                embedding_matrix[index] = unk_list
            else:
                embedding_matrix[index] = word2vec_matrix[word].tolist()
            used_index.append(index)

In [36]:
del used_index
gc.collect()

20

In [37]:
max_len_train = max([len(i) for i in word_index_train_list])
max_len_dev = max([len(i) for i in word_index_dev_list])
max_len_test = max([len(i) for i in word_index_test_list])

In [38]:
if max_len_train > max_len_test and max_len_train > max_len_dev:
    max_len = max_len_train
elif max_len_dev > max_len_train and max_len_dev > max_len_test:
        max_len = max_len_dev
else:
    max_len = max_len_test

In [39]:
# max_len_train,max_len_dev,max_len_test

# パディング

## 後ろpadding

In [40]:
index_for_padding = len(embedding_matrix)-1

In [41]:
def padding_index_list(num):
    return [index_for_padding] * num

In [42]:
copy_index_train_list = copy.deepcopy(word_index_train_list)

for index,word_index_list in enumerate(copy_index_train_list):
    pad_num = max_len - len(word_index_list)
    word_index_train_list[index].extend(padding_index_list(pad_num))

In [43]:
copy_index_dev_list = copy.deepcopy(word_index_dev_list)

for index,word_index_list in enumerate(copy_index_dev_list):
    pad_num = max_len - len(word_index_list)
    word_index_dev_list[index].extend(padding_index_list(pad_num))

## paddingされたものの場所をあらわすone-hot-vector

In [44]:
row_train = len(word_index_train_list[0])
row_dev = len(word_index_dev_list[0])

column = 1

batch_train = len(word_index_train_list)
batch_dev = len(word_index_dev_list)

In [45]:
#(barch,row,column)のlistを0で初期化
paded_one_hot_vector_train_list = [[[0.0] * column for i in range(row_train)] for j in range(batch_train)]
word_len_train_list = [len(i) for i in mecab_process_words_train_list]

paded_one_hot_vector_dev_list = [[[0.0] * column for i in range(row_dev)] for j in range(batch_dev)]
word_len_dev_list = [len(i) for i in mecab_process_words_dev_list]


In [46]:
del mecab_process_words_train_list
del mecab_process_words_dev_list
gc.collect()

20

In [47]:
#paddingされた所は0,されていない所は1のlist

for index,word_len in enumerate(word_len_train_list):
    
    #padding以降を1に変換
    #後ろpaddingを想定
    for i in range(word_len):
        paded_one_hot_vector_train_list[index][i] = [1.0] * column

for index,word_len in enumerate(word_len_dev_list):
    
    #padding以降を1に変換
    #後ろpaddingを想定
    for i in range(word_len):
        paded_one_hot_vector_dev_list[index][i] = [1.0] * column

In [48]:
paded_one_hot_vector_train_array = np.array(paded_one_hot_vector_train_list)
paded_one_hot_vector_dev_array = np.array(paded_one_hot_vector_dev_list)

In [49]:
del paded_one_hot_vector_train_list
del paded_one_hot_vector_dev_list

gc.collect()

60

# モデルの構築

In [50]:
time_steps = max_len
input_dim = size

## attention

In [51]:
class Attention_layer(Layer):
    
    def __init__(self, init='glorot_uniform', **kwargs):
        super(Attention_layer, self).__init__(**kwargs)
        self.supports_masking = True
        self.init = initializers.get(init)

    def build(self, input_shape):
        # input_shape = [(batch_size, input_dim, time_steps),(batch_size, input_dim, time_steps)]
        #前がlstm_out,後ろがmask
        assert len(input_shape[0]) == 3
        
        input_dim= input_shape[0][-1]
        batch_size = input_shape[0][0]

        self.kernel = self.add_weight(
            shape=(input_dim,1),
            name='kernel',
            initializer=self.init,
            trainable=True)
        
        self.W = self.add_weight(
        shape=(input_dim,input_dim),
        name='W',
        initializer=self.init,
        trainable=True)
        
        self.b = self.add_weight(
        shape=(input_dim),
        name='bias',
        initializer='zero',
        trainable=True)

        super(Attention_layer, self).build(input_shape)

    def call(self, x, mask=None):
       
        lstm_out = x[0]
        mask = x[1]
        
        ui = K.dot(lstm_out,self.W) 
        ui += self.b
        ui = K.tanh(ui)
        
        dot = K.dot(ui,self.kernel)
        
        
        multData = K.exp(dot)
        #print(K.int_shape(K.dot(self.kernel,x)))
        
        mask = Lambda(lambda x: tf.multiply(x[0],x[1]),name='masking')([mask,multData])
        
        output = mask/(K.sum(mask, axis=1)+K.epsilon())[:, None]
       
       # return K.reshape(output, (output.shape[0],output.shape[1],1))
        
        return output

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[1],1)

## LSTM

In [52]:
def model_attention_applied_after_lstm():
    #重みを固定する
    seed(1)
    inputs = Input(shape=(time_steps,), name='main_input')
    mask_input = Input(shape=(time_steps,1,),name='mask_input')
    
    embedding = Embedding(
        input_dim = embedding_matrix.shape[0],
        output_dim = embedding_matrix.shape[1],
        weights = [embedding_matrix],
        trainable = False,
        name = 'embedding')(inputs)

    #Dropout
    embedding = Dropout(DROPOUT)(embedding) 

    lstm_out = Bidirectional(
        LSTM(LSTM_UNITS,
             return_sequences=True,
             activation='tanh',
             name='biLSTM'),
        merge_mode='ave')(embedding)
    
    attention = Attention_layer(name='attention')([lstm_out,mask_input])

    attention_mul = Lambda(
        lambda x: x[0]*x[1],
        name='attention_mul')([lstm_out, attention])

    attention_merge = Lambda(
        lambda x: K.sum(x, axis=1),
        output_shape=lambda x: (x[0], x[2]),
        name="attention_sum")(attention_mul)
    
    output = Dense(1, name='regression')(attention_merge)


    model = Model(inputs=[inputs,mask_input], output=output)
    adam = optimizers.Adam()
    model.compile(loss='mean_squared_error', optimizer=adam)
    return model

In [53]:
#model_plot = model_attention_applied_after_lstm()
#plot_model(model_plot,show_shapes=True,to_file='/home/okazaki/time_measure/data_file/result_file/model.png')

# データサイズ

In [79]:
train_recipe_size = 10000
valid_recipe_size = 5000


In [80]:
word_index_train_nparray = np.array(word_index_train_list)

In [81]:
word_index_dev_nparray = np.array(word_index_dev_list)

In [82]:
X_train = word_index_train_nparray[:train_recipe_size]
X_valid = word_index_dev_nparray[:valid_recipe_size]

Y_train = log_time_array_train[:train_recipe_size]
Y_valid = log_time_array_dev[:valid_recipe_size]

mask_train = paded_one_hot_vector_train_array[:train_recipe_size]
mask_valid = paded_one_hot_vector_dev_array[:valid_recipe_size]

In [58]:
# word_index_train_list[0]

In [61]:
del word_index_train_nparray
del word_index_dev_nparray

del log_time_array_train
del log_time_array_dev

del paded_one_hot_vector_train_array
del paded_one_hot_vector_dev_array

gc.collect()

180

# fit

In [83]:
time_dev_list = df_id_process_time_turn_divide_time_dev['time'].tolist()
time_dev_list = time_procedure_to_process(time_dev_list,max_turn_dev_list)
a1=(math.log(5)+math.log(10))/2
a2=(math.log(10)+math.log(15))/2
a3=(math.log(15)+math.log(30))/2
a4=(math.log(30)+math.log(60))/2

In [84]:
def strict_acc(pred_dev):
    c5 = 0
    c10 = 0
    c15 = 0
    c30 = 0
    c60 = 0
    for ans_time, pred_time in zip(time_dev_list, pred_dev):
        if ans_time == 5:
            if pred_time <= a1:
                c5 += 1
        elif ans_time == 10:
            if a1 < pred_time <= a2:
                c10 += 1
        elif ans_time == 15:
            if a2 < pred_time <= a3:
                c15 += 1
        elif ans_time == 30:
            if a3 < pred_time <= a4:
                c30 += 1
        elif ans_time == 60:
            if a4 < pred_time:
                c60 += 1
 
    return (c5+c10+c15+c30+c60)/len(pred_dev)

In [ ]:
model = model_attention_applied_after_lstm()
y_train = Y_train

count = 0
before_loss = 100
best_loss = 100
save_path = '/hoge/hoge.hdf5'

j = 0
max_ACC = 0
seed(1)
# trainingにあるレシピをROUND回ループさせる
while(1):
    print('######################')
    j += 1
    print('%d回目' % j)
    
    his = model.fit([X_train, mask_train], y_train,
                    batch_size=BATCH_SIZE, epochs=1, verbose=1)

    train_loss = his.history['loss'][0]
    if best_loss > train_loss:
        best_loss = train_loss
        count = 0
    else:
        count += 1
    # best_lossが10回連続更新されなかったらbreak
    if count >= 10:
        break 
    
    
    pred_dev = model.predict([X_valid, mask_valid]).flatten()
    acc_dev = strict_acc(pred_dev)
   

    # best_modelの保存
    if max_ACC < acc_dev:
        json_string = model.to_json()
        model.save(save_path)
        y_train_best = y_train
        jj = j
        # 最小のMSEを変更
        max_ACC = acc_dev

    print('acc_dev=%f %d回目:max_ACC=%f' % (acc_dev, jj, max_ACC))